# Automated feature detection and classification of Electrocardiographic Time Series data - Notebook Implementation

## Cell 1 - Library imports

This cell will import all the libraries necessary for the machine learning pipeline. More libraries will be imported throughout the project for specific tasks but these libraries will be used for many of the tasks.

In [ ]:
import os
import numpy as np
import scipy.io
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import pywt
from pywt import wavedec

## Cell 2 - Empty DataFrame

An empty pandas dataframe will be created, using the input columns which would be filled using the ECG database [1].

In [ ]:
database = pd.DataFrame(columns=['condition', 'name', 'ecg'])

## Cell 3 - Data Upload

A Python module 'FileWizard' has been written that will upload all the data to the empty dataframe.
Database details:

The data contains 1000 ECG fragments of sample size 3600, there are 17 different heart rhythms in the dataset

Conditions:

* 1. NSR, Normal Sinus Rhythm [283 items]
* 2. APB, Atrial Premature Beat [66 items]
* 3. AFL, Atrial Flutter [20 items]
* 4. AFIB, Atrial Fibrilation [135 items]
* 5. SVTA, Superventricular tachycardia (abnormally fast heart beat above ventricular area) [13 items]
* 6. WPW, Wolf-Parkinson White Syndrome [21 items]
* 7. PVC, Premature Ventricular Contraction [133 items]
* 8. Bigeminy [55 items]
* 9. Trigeminy [13 items]
* 10. VT, Ventricular Tachycardia [10 items]
* 11. IVR, Idioventricular Rhythm [10 items]
* 12. VFL, Ventricular Flutter [10 items]
* 13. Fusion [11 items]
* 14. LBBB, Left Bundle Branch Block [103 items]
* 15. RBBB, Right Bundle Branch Block [62 items]
* 16. SDHB, Succinate dehydrogenase mutation [10 items]
* 17. PR, Pulmonary Valve Reguritation [45 items]

In [ ]:
from libraries.io import FileWizard

path1 = 'E:/uni work/FYP/code/database/MLII/'

fw = FileWizard()
database = fw.start(path1, database)

## Cells 4, 5 and 6 - Database testing status

Database records will be checked to see if all data has been uploaded, in the correct columns, and that all the conditions are correctly present.

In [ ]:
print(database.head(5))

In [ ]:
print('Records in database = ' + str(database['name'].size))

In [ ]:
print(database['condition'].unique())

## Cell 7 - ECG exploration

A healthy ECG heartbeat sample from the data is selected and explored. The SciPy library function find_peaks can be used to find the local maxima and minima, to find P, Q, R, S and T. This will form the basis of feature extraction later on in the project.

In [ ]:
case = '1 NSR'
examples = database[database['condition'] == case]
example = examples.iloc[1]
example_ecg = example['ecg']
eecg = example_ecg[160:455]
peaks, _ = find_peaks(eecg, height=960)
lowpeaks, _ = find_peaks(-eecg, height=-910)

plt.figure(figsize=(10, 6))
plt.title("Normal Sinus Rhythm Signal")
plt.xlabel("Samples")
plt.ylabel("Electrical Activity")
plt.plot(example_ecg[160:455])
plt.plot(peaks[2], eecg[peaks[2]], 'x')
plt.plot(peaks[4], eecg[peaks[4]], 'x')
plt.plot(lowpeaks[0], eecg[lowpeaks[0]], 'x')
plt.plot(lowpeaks[1], eecg[lowpeaks[1]], 'x')
plt.plot(peaks[5], eecg[peaks[5]], 'x')
plt.annotate('P = (' + str(peaks[2],) + ', ' + str(eecg[peaks[2]]) + ')', xy =(peaks[2] - 10, eecg[peaks[2]] + 10))
plt.annotate('Q = (' + str(lowpeaks[0],) + ', ' + str(eecg[lowpeaks[0]]) + ')', xy =(lowpeaks[0] - 50, eecg[lowpeaks[0]]))
plt.annotate('R = (' + str(peaks[4],) + ', ' + str(eecg[peaks[4]]) + ')', xy =(peaks[4] - 50, eecg[peaks[4]]))
plt.annotate('S = (' + str(lowpeaks[1],) + ', ' + str(eecg[lowpeaks[1]]) + ')', xy =(lowpeaks[1] + 5, eecg[lowpeaks[1]]))
plt.annotate('T = (' + str(peaks[4],) + ', ' + str(eecg[peaks[5]]) + ')', xy =(peaks[5] - 20, eecg[peaks[5]] + 10))

## Cell 8 - ECG comparison

A brief ECG comparison between NSR and AFL. There are identifable temporal and amplitudinal differences in quality between the samples. The differences between temporal and amplitudinal quality can be measured and used as a feature of the signal.

In [ ]:
case = '1 NSR'
examples = database[database['condition'] == case]
example = examples.iloc[1]
example_ecg = example['ecg']

case = '3 AFL'
examples = database[database['condition'] == case]
example = examples.iloc[1]
example_ecg2 = example['ecg']

fig, ((ax1, ax2)) = plt.subplots(2, figsize = (10,8), sharex=True, sharey=True)

ax1.title.set_text("Normal Sinus Rhythm")
ax1.set_ylabel("Electrical Activity")
ax1.set_xlabel("Samples")
ax1.plot(example_ecg[0:1200])
ax2.title.set_text("Atrial Flutter")
ax2.set_ylabel("Electrical Activity")
ax2.set_xlabel("Samples")
ax2.plot(example_ecg2[0:1200])

## Cell 9 - Multi ECG plotting method

Method used for plotting a selected piece of data from an ECG record or database. The method takes in the database, the condition, title desired for the plot, the record index desired, and what part of the record should be plotted. This method will be essential for multiple ECG data comparison. The method then queries the database for ECGs where the condition equals the input "case", then using the result of this query, record "i" would be obtained. Then to get a specific data sample from that record, the data from the desired column is selected.

In [ ]:
def plot_example(case, data, title, i, column):
    examples = data[data['condition'] == case]
    example = examples.iloc[i]
    example_ecg = example[column]
    
    plt.figure(figsize=(15,4))
    plt.title(title)
    plt.xlabel("Samples")
    plt.ylabel("Electrical Activity")
    plt.plot(example_ecg)
    plt.show()
    
lists = ['1 NSR', '2 APB', '3 AFL', '4 AFIB', '5 SVTA', '6 WPW', 
         '7 PVC', '8 Bigeminy', '9 Trigeminy', '10 VT', '11 IVR', 
         '12 VFL', '13 Fusion', '14 LBBBB', '15 RBBBB', '16 SDHB', '17 PR']

## Cell 10 - Multi ECG comparison

Using the method, a for loop will plot all ECGs for heart condition classes. For every case record 1 of every case will be plotted. This will be used to view the full dataset and to expect what the heart conditions would typically look like

In [ ]:
for item in lists:
    plot_example(item, database, (item + ' as raw data'), 1, 'ecg')

## Cell 11 - Noise Removal

A simplified method to remove baseline noise artefacts in ECGS [2] was devised by Balasubramanian, S. V. This method was used to remove baseline noise, and to re-scale the ECGs so the values are normalized. Also a Savitzky Golay filter was simultaneously used to remove phase noise from the ECGs.

In [ ]:
from libraries.noise_removal import BaselineNoiseRemover

# DC Notch filter to remove baseline noise from all signals

bnr = BaselineNoiseRemover(c = -0.99)

ecg_waves = database['ecg'].tolist()
ecg_filt = []

for wave in ecg_waves:
    filt = bnr.fit(wave)
    ecg_filt.append(filt)

database['ecg'] = pd.Series(ecg_filt)

## Cell 12 - Result of Noise Removal

Using the same method as before, all records will be plotted to view the result of baseline noise removal.

In [ ]:
for item in lists:
    plot_example(item, database, (item + ' with baseline noise removed'), 0, 'ecg')

## Cell 13 - Daubechies 4 Wavelet
Demonstration of Daubechies 4 wavelet. This will be used for the Discrete Wavelet Transform because the wavelet looks most identical to the ECG wave.

In [ ]:
wavelet = pywt.Wavelet('db4')
phi, psi, x = wavelet.wavefun(level=4)
plt.plot(x, psi)
plt.title('Daubechies 4')

## Cell 14 - Frequency Separation of ECGs.

In order to perform feature extraction effectively the PQRST components must be separated from each other. This is to prevent the threshold-based peak detection method from coming across any artefacts when locating the position of ECG components. The Frequency Extractor module uses DWT to decompose an ECG, and inverse DWT to recompose high frequency components of the ECG only, to retrieve the QRS complex frequencies. A novel step detection method also developed extracts P and T frequencies and stores them in the database.

In [ ]:
from libraries.feature_extraction import FrequencyExtractor

fe = FrequencyExtractor()
database = fe.fit(database)
# Multilevel discrete decomposition to extract large frequencies from time series

## Cells 15, 16 and 17 - Frequency Separation Results and Removal of Low Quality Data

A demonstration of the results of frequency extraction. Both large and small frequencies will be plotted here. The DWT method was successful in producing high quality QRS complex data ready for separation. However, because the quality of the P and T data is lacking, it is very difficult to extract notable features from the data.

The two conditions 10 VT and 12 VFL contain extremely noisy data in their ECGs, and thus have no real visible interval, so they will be ruled out.

In [ ]:
for item in lists:
    plot_example(item, database, (item + ' large frequencies'), 10, 'large frequencies')

In [ ]:
for item in lists:
    plot_example(item, database, (item + ' small frequencies'), 1, 'small frequencies')

In [ ]:
remove = ['10 VT', '12 VFL']

for item in remove:
    database = database[database['condition'] != item]

## Cell 18 - Distribution Comparison

To create as much feature data as possible for optimal machine learning results, there should be feature data for every heartbeat. A method will now be explored and designed to see what way the heartbeats can be separated. First of all, the distributions between two different QRS samples of the same condition are compared. As observable, they are out of phase with each other, which means a fixed window to separate the heartbeats would not be sufficient.

In [ ]:
import seaborn as sns

examples = database[database['condition'] == '1 NSR']
examples2 = database[database['condition'] == '1 NSR']
example1 = examples.iloc[1]
example2 = examples2.iloc[2]

ecg1 = example1['large frequencies']
ecg2 = example2['large frequencies']

plt.figure(figsize=(15,4))
plt.plot(ecg1)
plt.plot(ecg2)
plt.title('Comparison of distributions between two Normal Sinus Rhythm waves')
plt.show()

## Cell 19 - Dynamic Time Warping

A well known algorithm called Dynamic Time Warping has been used often to measure the difference in distribution between two time series data, I have developed my own implementation of the algorithm and it will be used to compare how similar in physical alignment the two ECGs above are. A distance matrix is calculated where the manhattan distance between each point of the two arrays are calculated, and then using the distance matrix, an optimal path will be found between the ECGs. If the path on the matrix is a straight line, that means that the speed of both signals are constant, and a fixed window can be used to separate the heartbeats. If not then a method must be developed that can separate heartbeats of various distances

In [ ]:
from libraries.distributions import DynamicTimeWarper

dtw = DynamicTimeWarper()

cost_matrix, path = dtw.fit(ecg1, ecg2)

## Cell 20 - Distance Matrix

As observable, the optimal pathway between the two samples is not directly proportional, and so a method must be developed.

In [ ]:
plt.figure(figsize=(16, 16))
ax = plt.axes()
sns.heatmap(cost_matrix, cmap='coolwarm_r', ax=ax)
ax.set_title("Dynamic Time Warping")
ax.set(xlabel='ECG 1', ylabel='ECG 2')

positions = path[1]
a = positions[:,1]
b = positions[:,0]
ax.plot(a + 0.5, b + 0.5, color='blue', linewidth=1)

## Cell 21 - Peak Extraction

A method to extract heartbeats from ECGs is to find the most prominent peaks, which are R peaks, then find the midpoint between every R peak which will be the separation point of the heartbeat. For conditions with a less prominent QRS complex a lower threshold is used.

This cell will extract the R peaks of every ECG using the SciPy find_peaks function.

In [ ]:
from libraries.feature_extraction import PeakExtractor

high_thresh = ['15 RBBBB', '17 PR']

db_low_thresh = database[database['condition'] == '11 IVR']
database = database[database['condition'] != '11 IVR']

thresh = 40
prominence = 50
pe = PeakExtractor(c=thresh, p=prominence)
database = pe.fit(database, 'large frequencies', 'peaks', 'peak position')

# Special threshold for 11 IVR signals, QRS complex is very small

thresh = 10
prominence = 25
pe = PeakExtractor(c=thresh, p=prominence)
db_low_thresh = pe.fit(db_low_thresh, 'large frequencies', 'peaks', 'peak position')

db_high_thresh = database[database['condition'].isin(high_thresh)]
database = database[~database['condition'].isin(high_thresh)]

thresh = 80
prominence = 50
pe = PeakExtractor(c=thresh, p=prominence)
db_high_thresh = pe.fit(db_high_thresh, 'large frequencies', 'peaks', 'peak position')

frames = [database, db_low_thresh, db_high_thresh]
database = pd.concat(frames)
database = database.reset_index()

## Cell 22 - Peak Extraction Test

This is a test to ensure that peaks were extracted properly. The database will be queried for a record, and the ECG, along with its peaks will be plotted.

In [ ]:
examples = database[database['condition'] == '17 PR']
example1 = examples.iloc[0]
peaks1 = example1['peaks']
ecg1 = example1['large frequencies']

plt.figure(figsize=(15,4))
plt.title('QRS peak detection of NSR')
plt.plot(ecg1)
plt.plot(peaks1, ecg1[peaks1], "x")
plt.plot(np.ones_like(ecg1) * thresh, "--", color="gray")
plt.show()

print(peaks1)
print(ecg1[peaks1])

## Cells 23 and 24 - Midpoint Extraction

Now the peaks are found for every ECG, the midpoint between every peak is calculated. The same record will be queried again from the database, this time to find midpoints of that ECG. This is how the heartbeats will be separated by.

In [ ]:
from libraries.feature_extraction import MidPointExtractor

mpe = MidPointExtractor()
database = mpe.fit(database, 'peaks', 'peak position', 'midpoints')

In [ ]:
examples = database[database['condition'] == '17 PR']
example1 = examples.iloc[0]
mid1 = example1['midpoints']
mid1 = mid1[0,:]

# Retrieval of the midpoints from the database record

plt.figure(figsize=(15,4))
plt.title('Optimal window detection of QRS complex via midpoint calculation')
plt.plot(ecg1)
for xv in mid1:
    plt.axvline(x=xv, color='black', linestyle='--')
plt.show()

## Cell 25 and 26 - Test on P and T

The test of the applicability of the wave separation method can be applied to the P and T waves, however the data quality produced was too low quality to extract features from. Unlike the QRS complexes which had consistent identifiable areas where features can be extracted from, these frequencies do not have enough identifiable characteristics or patterns that can be detected through simple thresholding. There are even some cases where some unremoved severe baseline noise remains in the data, which causes the P and T waves to rise above or below the threshold for peak detection, and then results in false positive peak detection or false negative peak detection.

In [ ]:
thresh = 5
prominence=0
pe = PeakExtractor(c=thresh, p=prominence)
database = pe.fit(database, 'small frequencies', 'small peaks', 'small peak position')

examples = database[database['condition'] == '2 APB']
example1 = examples.iloc[17]
peaks2 = example1['small peaks']
pt1 = example1['small frequencies']

# Retrieval of database for the peaks and their positions

plt.figure(figsize=(15,2))
plt.title('P/T peak detection of NSR signal')
plt.plot(pt1)
plt.plot(peaks2, pt1[peaks2], "x")
plt.plot(np.ones_like(pt1) * thresh, "--", color="gray")
plt.show()

In [ ]:
database = mpe.fit(database, 'small peaks', 'small peak position', 'small midpoints')

examples = database[database['condition'] == '2 APB']

example1 = examples.iloc[17]
mid2 = example1['small midpoints']
mid2 = mid2[0,:]

# Retrieval of the midpoints from the database record

plt.figure(figsize=(15,2))
plt.title('Optimal window detection of P and T signal via midpoint calculation')
plt.plot(pt1)
for xv in mid2:
    plt.axvline(x=xv, color='black', linestyle='--')
plt.show()

## Cell 27 - Heartbeat Separation

Once midpoints are found in every QRS sample, they are separated into heartbeats, so that they can have features extracted. These separate heartbeats are then inserted into a new database.

In [ ]:
from libraries.feature_extraction import WaveletSeparator

qrs_db = pd.DataFrame(columns=['wavelet', 'condition', 'partof'])

ws = WaveletSeparator()
qrs_db = ws.fit(database, qrs_db)

## Cells 28 and 29 - Heartbeat Separation Results

To test if the heartbeats were correctly separated, an ECG sample is queried from the first database, then from the second database, the first heartbeat from the sample is compared to the whole sample to see if the wave is correctly aligning with each other. This will also be tested for another condition, Atrial Flutter to see if the task is able to do the same for other conditions.

In [ ]:
fig, ((ax1, ax2, ax3)) = plt.subplots(3, figsize = (10,8), sharex=True, sharey=True)

examples = database[database['condition'] == '1 NSR']
examples1 = examples.iloc[0]
ecg1 = examples1['large frequencies']

examples1r = qrs_db[qrs_db['condition'] == '1 NSR']
examples1r = examples1r.iloc[0]
wav = examples1r['wavelet']

ax1.title.set_text("101m (0) NSR sample from database")
ax1.set_ylabel("Electrical Activity")
ax1.plot(ecg1)
ax2.title.set_text("101m (0) heartbeat from new database")
ax2.set_ylabel("Electrical Activity")
ax2.plot(wav)
ax3.title.set_text("Comparison")
ax3.plot(ecg1)
ax3.plot(wav)

In [ ]:
examples1 = qrs_db[qrs_db['condition'] == '5 SVTA']
examples2 = database[database['condition'] == '5 SVTA']

examples1r = examples1.iloc[0]
examples2r = examples2.iloc[0]

wav2 = examples1r['wavelet']
ecg2 = examples2r['large frequencies']

fig, ((ax1, ax2, ax3)) = plt.subplots(3, figsize = (10,8), sharex=True, sharey=True)

ax1.title.set_text("202m (1) Atrial Flutter Rhythm from database")
ax1.set_ylabel("Electrical Activity")
ax1.plot(ecg2)
ax2.title.set_text("202m (1) Atrial Flutter heartbeat from new database")
ax2.plot(wav2)
ax3.set_xlabel("Samples")
ax3.title.set_text("Comparison")
ax3.plot(ecg2)
ax3.plot(wav2)

## Cell 30 - Full Heartbeat Exploration

The piece of code below can be used to explore the full results of the heartbeat separation task. Most heartbeats have been separated correctly, because the use of peak prominence and threshold adjustment can be manually adjusted depending on the condition of the ECG. The "examples_iloc" index value can be changed to any desired index number to view the results.

In [ ]:
list_temp = ['1 NSR', '2 APB', '3 AFL', '4 AFIB', '5 SVTA', '6 WPW', 
         '7 PVC', '8 Bigeminy', '9 Trigeminy', '11 IVR', '13 Fusion', '14 LBBBB', '15 RBBBB', '16 SDHB', '17 PR']

for item in list_temp:
    examples = qrs_db[qrs_db['condition'] == item]
    ewav = examples.iloc[10]
    wav = ewav['wavelet']
    plt.title("Single wave of " + item)
    plt.ylabel("Electrical Activity")
    plt.plot(wav)
    plt.show()

## Cells 31 and 32 - Feature Extraction Design

These graphical illustrations take an example of the heartbeat and then is used to show what type of features are extracted from this heartbeat.

In [ ]:
examples1 = qrs_db[qrs_db['condition'] == '1 NSR']
ewav1 = examples1.iloc[1]
wav1 = ewav1['wavelet']

p1, pos1 = find_peaks(wav1, 25)
p2, pos2 = find_peaks(-wav1, 25)
print(wav1[p1])
pos1 = pos1['peak_heights']
pos2 = pos2['peak_heights']


p3 = p2[1]
p2 = p2[0]

plt.figure(figsize=(15,4))
plt.title('Anatomy of the QRS (NSR Example)')
plt.plot(wav1)
plt.plot(p1, wav1[p1], 'x', color='black')
plt.plot(p2, wav1[p2], 'x', color='black')
plt.plot(p3, wav1[p3], 'x', color='black')
w22 = -wav1[p2]
print(wav1[p1])
print(p1)
print(w22)
print(wav1[p1] + w22)

string1 = ('coordinates Q: ' + str(p1) + ', ' + str(wav1[p1]))
string2 = ('coordinates R: ' + str(p2) + ', ' + str(wav1[p2]))
string3 = ('coordinates S: ' + str(p3) + ', ' + str(wav1[p3]))
print(string1)
print(string2)
print(string3)
plt.annotate(('S = (147, -67.5)'), xy =(p3 + 5, wav1[p3]))
plt.annotate('Q = (127, -56.7)', xy =(p2 - 20, wav1[p2] - 10))
plt.annotate('R = (135, 81.1)', xy =(p1 + 5, wav1[p1] - 8))
plt.show()

plt.figure(figsize=(15,4))
plt.title('Height between Q and R')
plt.plot(wav1)
plt.plot(p1, wav1[p1], 'x', color='black')
plt.plot(p2, wav1[p2], 'x', color='black')
w22 = -wav1[p2]
print(wav1[p1])
print(w22)
print(wav1[p1] + w22)
plt.axhline(wav1[p1], xmin=0.43, xmax=0.57, color='grey', linestyle='--')
plt.axhline(wav1[p2], xmin=0.43, xmax=0.57, color='grey', linestyle='--')
plt.axvline(130, ymin=0.1, ymax=0.95, color='grey', linestyle='--')
plt.annotate('g = 137.8', xy =(70, 50))
plt.annotate('Q = (127, -56.7)', xy =(p2 - 20, wav1[p2] - 10))
plt.annotate('R = (135, 81.1)', xy =(p1 + 3, wav1[p1] - 8))
plt.show()

plt.figure(figsize=(15,4))
plt.title('Height between R to S')
plt.plot(wav1)
plt.plot(p1, wav1[p1], 'x', color='black')
plt.plot(p3, wav1[p3], 'x', color='black')
w22 = -wav1[p3]
print(wav1[p1])
print(w22)
print(wav1[p1] + w22)
plt.axhline(wav1[p1], xmin=0.43, xmax=0.57, color='grey', linestyle='--')
plt.axhline(wav1[p3], xmin=0.43, xmax=0.57, color='grey', linestyle='--')
plt.axvline(130, ymin=0.05, ymax=0.95, color='grey', linestyle='--')
plt.annotate('g = (81.1 - -67.5) = 148.6', xy =(70, 50))
plt.annotate('S = (147, -67.5)', xy =(p3 + 8, wav1[p3] - 1))
plt.annotate('R = (135, 81.1)', xy =(p1 + 3, wav1[p1] - 8))
plt.show()

plt.figure(figsize=(15,4))
plt.title('Time gap between Q and S')
plt.plot(p2, wav1[p2], 'x', color='black')
plt.plot(p3, wav1[p3], 'x', color='black')
plt.plot(wav1)
plt.axhline(0, xmin=0.46, xmax=0.54, color='grey', linestyle='--')
plt.axvline(p2, ymin=0.05, ymax=0.95, color='grey', linestyle='--')
plt.axvline(p3, ymin=0.05, ymax=0.95, color='grey', linestyle='--')
plt.annotate('Time length = 0.05 seconds', xy =(60, 50))
plt.annotate('S = (147, -67.5)', xy =(p3 + 8, wav1[p3] - 1))
plt.annotate('Q = (127, -56.7)', xy =(p2 - 20, wav1[p2] - 10))
plt.show()

plt.figure(figsize=(15,4))
plt.title('Time Length')
plt.plot(wav1)
w22 = -wav1[p3]
print(wav1[p1])
print(w22)
print(wav1[p1] + w22)
plt.axhline(0, xmin=0.05, xmax=0.95, color='grey', linestyle='--')
plt.axvline(0, ymin=0.05, ymax=0.95, color='grey', linestyle='--')
plt.axvline(len(wav1), ymin=0.05, ymax=0.95, color='grey', linestyle='--')
plt.annotate('Time length = 0.7 seconds', xy =(90, 50))
plt.show()

In [ ]:
wav1 = examples1.iloc[31]
wav1w = wav1['wavelet']
wav11 = examples1.iloc[32]
wav11w = wav11['wavelet']
demo = np.hstack((wav1w, wav11w))
high, pos1 = find_peaks(demo, height=50)
low, pos2 = find_peaks(-demo, height=30, prominence=6)

r1 = high[0]
r2 = high[1]

plt.figure(figsize=(15,4))
plt.title('D1 and D2')
plt.plot(demo)
plt.show()

print(r1)
print(demo[r1])
print(r2)
print(demo[r2])

print((r2- r1) * 0.0027777777)

plt.figure(figsize=(15,4))
plt.title('Time gap between two intervals')
plt.plot(demo)
plt.plot(r1, demo[r1], 'x', color='black')
plt.plot(r2, demo[r2], 'x', color='black')
plt.annotate('Time gap length = 0.71 seconds', xy =(200, 50))
plt.annotate('R1 = (128, 82.9)', xy =(r1 - 65, demo[r1] - 5))
plt.annotate('R2 = (383, 88.3)', xy =(r2 - 65, demo[r2] - 5))
plt.axvline(r1, ymin=0.05, ymax=0.95, color='black', linestyle='--')
plt.axvline(r2, ymin=0.05, ymax=0.95, color='black', linestyle='--')
plt.axhline(0, xmin=0.28, xmax=0.71, color='black', linestyle='--')
plt.show()

## Cells 33, 34, 35, and 36 - Full Feature Extraction

These features are then extracted from the entire dataset of heartbeats. A module that I have developed in Python uses the same QRS detection techniques to fill the database with scalar numbered features relating to the amplitudinal and temporal composition of the heartbeat. Once this feature extraction process has completed, then in the case of any NaN values due to anomalous data having an effect on feature extraction, this can be removed using "dropna()".

A backup database called "qrs_db_backup" has been created in the event of data corruption that is based on the original database. Run Cell 34 in case of this happening.

In [ ]:
qrs_db_backup = qrs_db

In [ ]:
qrs_db = qrs_db_backup

In [ ]:
from libraries.qrs_features import QRSfeatures

qrsf = QRSfeatures(50)
qrs_db= qrsf.fit(qrs_db)

In [ ]:
qrs_db = qrs_db.dropna()
qrs_db = qrs_db.reset_index(drop=True)

## Cells 37 and 38 - Box Plot of Features

This is the results of the feature extraction, from the boxplots of the five features, the distributions of the five features can be viewed here. The significance of the box plot is to see how distinct the feature values make the conditions when it comes to classification.

In [ ]:
listdata = ['1 NSR', '2 APB', '3 AFL', '4 AFIB', '5 SVTA', '6 WPW', 
         '7 PVC', '8 Bigeminy', '9 Trigeminy', '11 IVR', '13 Fusion', 
            '14 LBBBB', '15 RBBBB', '16 SDHB', '17 PR']

def box(data, label, time_mode):
    
    list3 = []
    
    for item in listdata:  
        subject = data[data['condition'] == item]
        lots = subject[label].to_list()
        col = pd.DataFrame(lots, columns=[item])
        list3.append(col)
    
    new_data = pd.concat(list3, axis=1)
    bp = new_data.boxplot(column=listdata, figsize=((15,10)), vert=False)
    
    if time_mode == True:
        bp.set_xlabel(label + ' (seconds)') 
    else:
        bp.set_xlabel(label + ' (mV)') 
    plt.show()

In [ ]:
box(qrs_db[['condition','time length']], 'time length', True)
box(qrs_db[['condition','qr height']], 'qr height', False)
box(qrs_db[['condition','rs height']], 'rs height', False)
box(qrs_db[['condition','qs time length']], 'qs time length', True)
box(qrs_db[['condition','time gap']], 'time gap', True)

## Cells 39, 40 and 41 - Removal of Outliers

Because it is very likely that outliers will be found in the data, then there needs to be a function that can remove values that are beyond the whiskers of every condition and the interquartile range. If outliers are not removed this can severely cause the feature space to become fuzzy, therefore hindering the accuracy of the machine learning algorithm.

In [ ]:
def remove_outliers(data):
    
    variables = ['time gap', 
             'qr height', 'rs height', 'qs time length', 'time length']

    for var in variables:
        cols = ['time gap', 'qr height', 'rs height', 'qs time length', 'time length']
        Q1 = data[cols].quantile(0.25)
        Q3 = data[cols].quantile(0.75)
        IQR = Q3 - Q1

        data = data[~((data[cols] < (Q1 - 1.5 * IQR)) |(data[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return data

In [ ]:
qrs_db_t = qrs_db

new_subjects = []

for cond in listdata:
    subject = qrs_db_t[qrs_db_t['condition'] == cond]
    new_subject = remove_outliers(subject)
    new_subjects.append(new_subject)
    
qrs_db_t = pd.concat(new_subjects)
qrs_db_t = qrs_db_t.reset_index(drop=True)

In [ ]:
box(qrs_db_t[['condition','time length']], 'time length', True)
box(qrs_db_t[['condition','qr height']], 'qr height', False)
box(qrs_db_t[['condition','rs height']], 'rs height', False)
box(qrs_db_t[['condition','qs time length']], 'qs time length', True)
box(qrs_db_t[['condition','time gap']], 'time gap', True)

## Cell 42 - Data Equalization

For optimal results, it is best that for every case the number of heartbeats are equalized. The database is edited so that for every case in the data the first 60 heartbeats are taken.

In [ ]:
def edit_data(data, listdata, maximum):  
    lists2 = []
    for item in listdata:  
        subject = data[data['condition'] == item]
        subject = subject[0:maximum]
        lists2.append(subject)
    
    new_data = pd.concat(lists2)
    new_data = new_data.reset_index(drop=True)
    
    return new_data

In [ ]:
qrs_db_t = edit_data(qrs_db_t, listdata, 200)

## Cells 43 and 44 - Linear Analysis of Features

To study how strong of a relationship two features have, a linear analysis of two variables can be undertaken. This measures how each condition is clustered and represented in the feature space. If a large majority of heart conditions cluster around a certain feature range on the two dimensional space, it shows that there can be a decision boundary established.

In [ ]:
def linear_analysis(data, labels, var1, var2):
    plt.figure(figsize=(15,15))
    for item in labels:
        subject = data[data['condition'] == item]
        plt.plot(subject[var1], subject[var2], 'x', label=str(item))
    
    plt.title('Feature space of heartbeats')
    plt.xlabel(var1)
    plt.ylabel(var2)
    plt.legend()
    plt.show()

In [ ]:
linear_analysis(qrs_db_t, listdata, 'time length', 'time gap')

## Cells 46 and 47 - Test 1 preparation

For the first machine learning test, only the two time related features, time length and time gap will be used. This is to test first of all how the machine learning performs with only one category of features, the temporal. If the machine learning model underperforms, then it is an indication that more features are necessary.

In [ ]:
def parse_labels(X, y, label):
    for i, item in enumerate(label):
        y = y.replace(label, i)
    y = y.astype("category")
    y = y.reset_index(drop='True')
    X = X.reset_index(drop='True')
    return X, y

In [ ]:
X = qrs_db_t[['time length', 'time gap']].astype(float)
y = qrs_db_t['condition']

X, y = parse_labels(X, y, listdata)

## Cells 48 and 49 - XGBoost Random Forest parameters

This method will be responsible for arranging the XGBoost parameters for every model. This is so the parameters do not have to be changed constantly with every test. To ensure a repeatable and reproducible test all parameters should be the same for every test.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def ml_model(X, y):
    params = {
        'colsample_bynode': 0.8,
        'learning_rate': 3,
        'max_depth': 6,
        'num_parallel_tree': 300,
        'subsample': 0.8,
        'tree_method': 'gpu_hist',
        'reg_lambda': 100
    }

    xgb = XGBClassifier(params)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, 
                                                        test_size=0.3, stratify=y, random_state=0)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
    return y_test, y_pred

## Cell 50 - Confusion Matrix

For every test a confusion matrix will be plotted in order to analyse where the machine learning models went wrong. By seeing what the model misclassified, the quality of the features as an input can potentially be used to justify the areas of confusion that a machine learning model may experience. This method written here plots the confusion matrix for the predictions made by the machine learning model vs the actual results.

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

def conmat(test, pred, labels):
    plt.figure(figsize=(10, 10))
    ax = plt.axes()
    mat = confusion_matrix(test_one_test, test_one_pred)
    sns.heatmap(mat, cmap='coolwarm_r', ax=ax, xticklabels=labels, yticklabels=labels, annot=True)
    plt.xlabel('Prediction')
    plt.ylabel('Actual')
    plt.show()

## Cell 51 - Classification Report

The classification report imported through sci-kit learn will be used to calculate the following for every model:

* Precision - the proportion of correctly predicted positives in the model against all total predictions that were considered positive by the model
* Recall - the proportion of correctly predicted positives in the model against the number of actual positive cases in the data
* F1-Score - the weighted average of precision and recall

In [ ]:
from sklearn.metrics import classification_report

## Cell 52 - Test 1

The running and results of the Test 1 model including confusion matrix and classification report.

In [ ]:
test_one_test, test_one_pred = ml_model(X, y)
conmat(test_one_test, test_one_pred, listdata)
print(classification_report(y_test, y_pred, target_names=listdata))

## Cells 53 and 54 - Test 2

Now a test using all of the features. This test will test the significance of using all of the data's features. If accuracy has increased, it shows that more features improve the decision boundaries of data prediction. However if accuracy decreases there may be an error in the data itself, or the choice of machine learning model is not suitable.

In [ ]:
X = qrs_db_t[['time length', 'time gap', 'qr height', 'rs height', 'qs time length']].astype(float)
y = qrs_db_t['condition']

X, y = parse_labels(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, stratify=y)

In [ ]:
test_one_test, test_one_pred = ml_model(X, y)
conmat(test_one_test, test_one_pred, listdata)
print(classification_report(y_test, y_pred, target_names=labels))

## Cells 55, 56, 57, 58, 59 and 60 - Test 3

The final test will not use the features for training but will instead use the input of the raw wavelets into the model. This indicate if the extraction of features was actually necessary for the project. The expected result is that the prediction would be fuzzier than the previous tests, because arrays have a far bigger space complexity than scalar feature variables. Because the input matrix must be of two dimensions, all the heartbeats will need to be padded so that every heartbeat is the same array size, and then all heartbeat arrays will be stacked to form a two dimensional matrix. This does not affect the physical composition of the waves.

In [ ]:
X = qrs_db_t['wavelet']
XL = qrs_db_t['time length']

y = qrs_db_t['condition']

In [ ]:
lengths = []
for a in X:
    lengths.append(len(a))
maximum = np.amax(lengths)
print(maximum)

In [ ]:
X_new = []
for x in X:
    x = np.resize(x, maximum)
    X_new.append(x)

In [ ]:
for a in X_new:
    print(len(a))

In [ ]:
X = np.asmatrix(X_new)

In [ ]:
test_one_test, test_one_pred = ml_model(X, y)
conmat(test_one_test, test_one_pred, listdata)
print(classification_report(y_test, y_pred, target_names=labels))

## References

1. P lawiak, P. [2017], ‘Ecg signals (1000 fragments)’.
2. Balasubramanian, S. V. [n.d.], ‘Power line interference and baseline removal in ecg’.